# Pregunta 3
## Jorge Schenke Larraín
### n°: 17641624

Lo primero que haremos será definir la clase Leaf la cual tendrá los siguientes atributois:
<ul>
    <li>hash (string)</li>
    <li>children (List(Leafs))</li>
    <li>parent (Leaf)</li>
</ul>

In [2]:
import hashlib

In [3]:
class Leaf:
    def __init__(self, hash):
        self.hash = hash
        self.children = []
        self.parent = None

Ahora que tenemos la clase Leaf, podemos definir la clase MerkleTree, sus atributos serán los siguientes:
<ul>
    <li>leafs (List(Leafs))</li>
    <li>hash (Function)</li>
    <li>root (Leaf)</li>
</ul>
Y contará con los siguientes métodos:
<ul>
    <li>self.parent(leaf1, leaf2): recibe 2 hojas y crea el padre a partir de los hashes de las hojas.</li>
    <li>self.create_tree(leafs): Crea el árbol a partir de sus hojas utilizando la función self.parent de forma recursiva.</li>
    <li>self.get_proof_for(string): Retorna una prueba de que cierto elemento es o no parte de las hojas del árbol</li>
    <li>self.display(node): Imprime todos los nodos descendientes desde el nodo entregado</li>
    <li>self.get_root(): retorna la raíz del nodo</li>
</ul>

In [10]:
class MerkleTree:
    def __init__(self, strings, hash_func):
        self.hash = hash_func
        leafs = []
        for string in strings:
            leaf = Leaf(self.hash(string))
            leafs.append(leaf)
        self.leafs = leafs
        self.root = Leaf
        self.create_tree(self.leafs)

    def parent(self, leaf1, leaf2):
        parent_hash = self.hash(leaf1.hash + leaf2.hash)
        parent_leaf = Leaf(parent_hash)
        leaf1.parent = parent_leaf
        leaf2.parent = parent_leaf
        parent_leaf.children = [leaf1, leaf2]
        return parent_leaf

    
    def create_tree(self, leafs):
        if len(leafs) > 1:
            parents = []
            par = (len(leafs) % 2 == 0)
            if not par:
                leafs.append(Leaf(leafs[-1].hash))
            i = 0
            while i < len(leafs):
                leaf1 = leafs[i]
                leaf2 = leafs[i+1]
                parents.append(self.parent(leaf1, leaf2))
                i += 2
            self.create_tree(parents)
        else:
            self.root = leafs[0]
            return self.root

    def get_proof_for(self, string):
        bros = None
        hash = self.hash(string)
        for leaf in self.leafs:
            if leaf.hash == hash:
                bros = []
                while leaf.parent is not None:
                    for child in leaf.parent.children:
                        if child != leaf:
                            place = leaf.parent.children.index(child)
                            if place == 0:
                                p = "i"
                            else:
                                p = "d"
                            bros.append((child.hash, p))
                    leaf = leaf.parent
                break
        
        return bros


    def display(self, node):
        print(f"Nodo:{node.hash}")
        if node.parent is not None:
            print(f"Parent: {node.parent.hash}")
        if len(node.children) > 0:
            print("Hijos:")
            for child in node.children:
                print(f"- {child.hash}")
            for child in node.children:
                self.display(child)
        else:
            print("Soy hoja")

    def get_root(self):
        print(f"root: {self.root.hash}")
        return self.root.hash

Ahora definamos la función verify:

In [11]:
def verify(root, string, proof, hash):
        next = hash(string)
        if proof is not None:
            for bro in proof:
                if bro[1] == "d":
                    next = hash(next + bro[0])
                else:
                    next = hash(bro[0] + next)
        if next == root:
            return True
        else:
            return False

Veamos un ejemplo sencillo:<br>
Como función de "hash" utilizaremos una función que retorna el string entregado

In [12]:
def hashe(string):
    return hashlib.md5(string.encode()).hexdigest()

Merkl = MerkleTree(["a", "b", "c"], hashe)

Mostremos el árbol completo:

In [13]:
Merkl.display(Merkl.root)

Nodo:d54ff7d0c3b219bcd40fc4107d0ab4f5
Hijos:
- 3bc22fb7aaebe9c8c5d7de312b876bb8
- 12730a37aa9d6d931096130616fead25
Nodo:3bc22fb7aaebe9c8c5d7de312b876bb8
Parent: d54ff7d0c3b219bcd40fc4107d0ab4f5
Hijos:
- 0cc175b9c0f1b6a831c399e269772661
- 92eb5ffee6ae2fec3ad71c777531578f
Nodo:0cc175b9c0f1b6a831c399e269772661
Parent: 3bc22fb7aaebe9c8c5d7de312b876bb8
Soy hoja
Nodo:92eb5ffee6ae2fec3ad71c777531578f
Parent: 3bc22fb7aaebe9c8c5d7de312b876bb8
Soy hoja
Nodo:12730a37aa9d6d931096130616fead25
Parent: d54ff7d0c3b219bcd40fc4107d0ab4f5
Hijos:
- 4a8a08f09d37b73795649038408b5f33
- 4a8a08f09d37b73795649038408b5f33
Nodo:4a8a08f09d37b73795649038408b5f33
Parent: 12730a37aa9d6d931096130616fead25
Soy hoja
Nodo:4a8a08f09d37b73795649038408b5f33
Parent: 12730a37aa9d6d931096130616fead25
Soy hoja


Busquemos la ráiz del árbol:

In [14]:
Merkl.get_root()

root: d54ff7d0c3b219bcd40fc4107d0ab4f5


'd54ff7d0c3b219bcd40fc4107d0ab4f5'

Busquemos una hoja que existe:

In [15]:
proof = Merkl.get_proof_for("c")
print(proof)

Aquitoy
[('4a8a08f09d37b73795649038408b5f33', 'd'), ('3bc22fb7aaebe9c8c5d7de312b876bb8', 'i')]


Ahora una inexistente:

In [16]:
not_proof = Merkl.get_proof_for("x")
print(not_proof)

None


Probemos la función verify con el ejemplo anterior:

In [17]:
print(verify("d54ff7d0c3b219bcd40fc4107d0ab4f5", "c", proof, Merkl.hash))

True


In [18]:
print(verify("abcc", "x", not_proof, Merkl.hash))

False


Podemos ver que la función cumple correctamente y es capaz de diferenciar pruebas válidas.